In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import BoxStyle

In [2]:
class Field(object):
    def __init__(self, name, default=0, help=None):
        self.name = name
        self.default = default
        self.help = help or ''


class Product(object):
    @classmethod
    def fields(cls):
        return []
    
    def __init__(self, **kwargs):
        fields = {f.name: f for f in self.fields()}
        self.fields = {f.name: f.default for f in fields.values()}
        for name, value in kwargs.items():
            if name not in fields:
                raise ValueError("Field '%s' is not recognized" % name)
            self.fields[name] = value
    
    def payoff(self, spot):
        raise NotImplementedError

        
class Option(Product):
    @classmethod
    def fields(cls):
        return [Field('strike')]


class Call(Option):
    def payoff(self, spot):
        return np.maximum(0.0, spot - self.fields['strike'])


class Put(Option):
    def payoff(self, spot):
        return np.maximum(0.0, self.fields['strike'] - spot)

    
class Collar(Product):
    @classmethod
    def fields(cls):
        return [Field('call_strike', default=30), Field('put_strike', default=10)]
    
    def payoff(self, spot):
        return Call(strike=self.fields['call_strike']).payoff(spot) \
            - Put(strike=self.fields['put_strike']).payoff(spot)

In [3]:
from functools import partial

import ipywidgets as widgets
from IPython.display import clear_output, display

products = [Collar, Call, Put]
products = {p.__name__: p for p in products}

plt.style.use('seaborn-whitegrid')

def plot_payoff(product, size, **kwargs):
    spots = np.arange(min(kwargs.values()) - size, max(kwargs.values()) + size)
    xlim = [min(spots), max(spots)]
    payoff_values = product(**kwargs).payoff(spots)

    fig = plt.figure(figsize=(14, 14))
    ax = plt.axes()
    ax.tick_params(labelsize='x-large')

    for key, value in kwargs.items():
        ax.axvline(x=value, color='black', linestyle='--', alpha=0.5, label=key, lw=2)
        plt.text(value, max(payoff_values), '%s = %s ' % (key, value),
                 rotation=90, va='top', ha='right', fontsize='x-large', color='black', alpha=0.5)

    ax.plot(spots, payoff_values, lw=3)
    ax.set_aspect('equal')
    ax.set_xlim(min(spots), max(spots))
    plt.title('Payoff function for a %s option' % product.__name__, size='xx-large')

    plt.show()

def show_widgets():
    setup = True
    field_widget = []

    def print_price(**kwargs):
        product = product_widget.value
        plot_payoff(product, 20, **kwargs)

    def product_changed(product):
        fields = {f.name: widgets.FloatText(description=f.name, value=f.default) for f in product.fields()}
        if field_widget:
            field_widget[0].close()
        new_i = widgets.interactive(print_price, **fields)

        display(new_i)
        field_widget[:] = [new_i]

    product_widget = widgets.Select(options=products)

    product_i = widgets.interactive(product_changed, product=product_widget)

    display(product_i)

    setup = False

show_widgets()